In [55]:
import pandas as pd 
from sqlalchemy import create_engine

In [56]:
user = "root"
password = "Olivia*23"
host = "localhost"
database = "sakila"

engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{database}")


In [57]:
import pandas as pd
import pymysql

def rentals_month(month: int, year: int) -> pd.DataFrame:
    """
    Retrieve rental data for a given month and year.
    Returns a pandas DataFrame with rental_id, customer_id, and rental_date.
    """
    conn = pymysql.connect(
        host="localhost",
        user="root",             # 👈 replace if different
        password="your_password", # 👈 replace with your MySQL pw
        database="sakila"
    )
    
    query = """
        SELECT rental_id, customer_id, rental_date
        FROM rental
        WHERE MONTH(rental_date) = %s
          AND YEAR(rental_date) = %s;
    """
    
    df = pd.read_sql(query, conn, params=[month, year])
    conn.close()
    return df


In [58]:
def rental_count_month(df: pd.DataFrame, month: int, year: int) -> pd.DataFrame:
    """
    Count number of rentals per customer in the given DataFrame.
    Returns a DataFrame with customer_id and rentals_{month}_{year} column.
    """
    col_name = f"rentals_{str(month).zfill(2)}_{year}"
    grouped = df.groupby("customer_id")["rental_id"].count().reset_index()
    grouped.rename(columns={"rental_id": col_name}, inplace=True)
    return grouped


In [59]:
def compare_rentals(df1: pd.DataFrame, df2: pd.DataFrame) -> pd.DataFrame:
    """
    Compare two months of rental counts.
    Returns a combined DataFrame with a 'difference' column.
    """
    merged = pd.merge(df1, df2, on="customer_id", how="inner")
    
    # Assumes only two rental columns + customer_id
    rental_cols = [col for col in merged.columns if col.startswith("rentals_")]
    if len(rental_cols) == 2:
        merged["difference"] = merged[rental_cols[1]] - merged[rental_cols[0]]
    return merged


In [60]:
may_rentals = rentals_month(5, 2005)
june_rentals = rentals_month(6, 2005)

print(may_rentals.head())
print(june_rentals.head())


OperationalError: (1045, "Access denied for user 'root'@'localhost' (using password: YES)")

In [ ]:
# Rentals in May 2005
may_rentals = rentals_month(engine, 5, 2005)
may_counts = rental_count_month(may_rentals, 5, 2005)

# Rentals in June 2005
june_rentals = rentals_month(engine, 6, 2005)
june_counts = rental_count_month(june_rentals, 6, 2005)

# Compare customers active in both months
comparison = compare_rentals(may_counts, june_counts)
comparison.head(20)  # show first 20 rows


TypeError: Cannot convert numpy.ndarray to numpy.ndarray